In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Загрузка данных
url = "https://axe.inline-ltd.ru/data/meatinfo.csv"
data = pd.read_csv(url, sep=';')

# Отфильтруем классы с менее чем 500 примерами
product_counts = data['mtype'].value_counts()
valid_products = product_counts[product_counts >= 500].index
data = data[data['mtype'].isin(valid_products)]

# Уменьшите размер данных для отладки
sample_data = data.sample(5000, random_state=42)  # Используем 1000 примеров для отладки

# Разделение данных на обучающую и тестовую выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(
    sample_data['text'], 
    sample_data['mtype'],
    test_size=0.2,
    random_state=42
)

# Кодирование меток
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)
num_classes = len(label_encoder.classes_)

c:\Users\rostyslav\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# Создание кастомного Dataset и DataLoader
class MeatDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Инициализация токенизатора и создание DataLoader
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
max_len = 128

train_dataset = MeatDataset(train_texts.values, train_labels, tokenizer, max_len)
test_dataset = MeatDataset(test_texts.values, test_labels, tokenizer, max_len)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [21]:
# Определение и обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=num_classes)
model = model.to(device)

# Оптимизатор
optimizer = AdamW(model.parameters(), lr=2e-5)

# Обучение модели
epochs = 4  # Уменьшите количество эпох для быстрой проверки

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\rostyslav\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4, Loss: 0.5189
Epoch 2/4, Loss: 0.1416
Epoch 3/4, Loss: 0.1174
Epoch 4/4, Loss: 0.1008


In [22]:
# Оценка модели
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

    Баранина       0.98      1.00      0.99        64
    Говядина       1.00      0.99      0.99       513
     Индейка       0.99      1.00      0.99        83
        Кура       0.85      0.72      0.78        92
     Свинина       0.98      0.99      0.98       198
    Цыпленок       0.60      0.76      0.67        50

    accuracy                           0.95      1000
   macro avg       0.90      0.91      0.90      1000
weighted avg       0.96      0.95      0.96      1000



In [23]:
# Тестирование на новых примерах
def predict(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, pred = torch.max(outputs.logits, dim=1)
    
    return label_encoder.inverse_transform(pred.cpu().numpy())[0]

examples = [
    "Говядина блочная 2 сорт в наличии ООО 'АгроСоюз' реализует блочную говядину 2 сорт (80/20) Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.",
    "Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:",
    "Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.",
]

for example in examples:
    print(f'Text: {example}\nPredicted Product: {predict(example)}\n')

Text: Говядина блочная 2 сорт в наличии ООО 'АгроСоюз' реализует блочную говядину 2 сорт (80/20) Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.
Predicted Product: Говядина

Text: Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:
Predicted Product: Кура

Text: Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.
Predicted Product: Говядина



Это чисто для меня

In [1]:
for example in zip(data['text'], data['mtype']):
    ## print(f'Text: {example[0]}\nPredicted Product: {predict(example[0])}\nExpected Product: {example[1]}\n')
    print(predict(example[0]) == example[1])

NameError: name 'data' is not defined